In [1]:
import pandas as pd
import csv
import os
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
import joblib
from config import user_name,password

In [2]:
btc = pd.read_csv("data/BTC-USD.csv")
btc = btc.drop(columns=["Adj Close","Open","High","Low","Volume"])
btc['Date']= pd.to_datetime(btc['Date'])
btc = btc.dropna()
btc_data = btc.iloc[::-1]
# btc_data.head()

In [3]:
btc_data = btc_data.rename(columns={'Close': 'Close_Btc'})
btc_data['Date'] = pd.to_datetime(btc_data['Date'])
btc_data['Day_Btc'] = btc_data['Date'].dt.day_name()
btc_data = btc_data.sort_values(by="Date")
# btc_data

In [4]:
btc_data_clean = btc_data.loc[((btc_data['Day_Btc'] != "Saturday") & (btc_data['Day_Btc'] != 'Sunday'))]
# btc_data_clean

In [5]:
# btc_data_clean = btc_data_clean.drop(columns=["Day_Btc"])
btc_data_clean["BTC_Change%"] = btc_data_clean["Close_Btc"].pct_change()
btc_data_clean['BTC_Change%_lag'] = btc_data_clean["BTC_Change%"].shift(1)

# Construction direction of BITCOIN movement
btc_data_clean['BTC_dir'] = (btc_data_clean["BTC_Change%"]>0).astype(float)
btc_data_clean['BTC_dir_lag'] = btc_data_clean['BTC_dir'].shift(1)
# btc_data_clean

/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a co

In [6]:
btc_data_clean.dtypes

Date               datetime64[ns]
Close_Btc                 float64
Day_Btc                    object
BTC_Change%               float64
BTC_Change%_lag           float64
BTC_dir                   float64
BTC_dir_lag               float64
dtype: object

In [7]:
eth = pd.read_csv("data/ETH-USD.csv")
eth = eth.drop(columns=["Adj Close","Open","High","Low","Volume"])
eth['Date']= pd.to_datetime(eth['Date'])
eth_data = eth.dropna()
eth_data = eth_data.iloc[::-1]
# eth_data.head()

In [8]:
eth_data = eth_data.rename(columns={'Close': 'Close_Eth'})
eth_data['Date'] = pd.to_datetime(eth_data['Date'])
eth_data['Day_Eth'] = eth_data['Date'].dt.day_name()
eth_data = eth_data.sort_values(by="Date")
# eth_data

In [9]:
eth_data_clean = eth_data.loc[((eth_data['Day_Eth'] != "Saturday") & (eth_data['Day_Eth'] != 'Sunday'))]
# eth_data_clean

In [10]:
# eth_data_clean = eth_data_clean.drop(columns=["Day_Eth"])
eth_data_clean["ETH_Change%"] = eth_data_clean["Close_Eth"].pct_change()
eth_data_clean['ETH_Change%_lag'] = eth_data_clean["ETH_Change%"].shift(1)

# Construction direction of Ethereum movement
eth_data_clean['ETH_dir'] = (eth_data_clean["ETH_Change%"]>0).astype(float)
eth_data_clean['ETH_dir_lag'] = eth_data_clean['ETH_dir'].shift(1)
eth_data_clean.head()

/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a co

,Date,Close_Eth,Day_Eth,ETH_Change%,ETH_Change%_lag,ETH_dir,ETH_dir_lag
2,2016-10-10,11.7555,Monday,NaN,NaN,0.0,NaN
3,2016-10-11,11.7857,Tuesday,0.002569,NaN,1.0,0.0
4,2016-10-12,11.9335,Wednesday,0.012541,0.002569,1.0,1.0
5,2016-10-13,11.9630,Thursday,0.002472,0.012541,1.0,1.0
6,2016-10-14,11.9382,Friday,-0.002073,0.002472,0.0,1.0


In [11]:
eth_data_clean.dtypes

Date               datetime64[ns]
Close_Eth                 float64
Day_Eth                    object
ETH_Change%               float64
ETH_Change%_lag           float64
ETH_dir                   float64
ETH_dir_lag               float64
dtype: object

In [12]:
gold = pd.read_csv("data/Gold-Futures.csv")
gold = gold.drop(columns=["Vol.","Change %","Open","High","Low"])
gold['Date']= pd.to_datetime(gold['Date'])
gold_data = gold.dropna()
# gold_data.head()

In [13]:
gold_data = gold_data.rename(columns={'Price': 'Price_Gold'})
gold_data['Date'] = pd.to_datetime(gold_data['Date'])
gold_data['Day_Gold'] = gold_data['Date'].dt.day_name()
gold_data = gold_data.sort_values(by="Date")
gold_data = gold_data.reset_index(drop=True)
gold_data['Price_Gold'] = gold_data.Price_Gold.str.replace(',', '').astype(float)
# gold_data

In [14]:
gold_data_clean = gold_data.loc[((gold_data['Day_Gold'] != "Saturday") & (gold_data['Day_Gold'] != 'Sunday'))]
# gold_data_clean

In [15]:
# gold_data_clean = gold_data_clean.drop(columns=["Day_Gold"])
gold_data_clean["Gold_Change%"] = gold_data_clean["Price_Gold"].pct_change()
gold_data_clean['Gold_Change%_lag'] = gold_data_clean["Gold_Change%"].shift(1)

# Construction direction of GOLD movement
gold_data_clean['gold_dir'] = (gold_data_clean["Gold_Change%"]>0).astype(float)
gold_data_clean['gold_dir_lag'] = gold_data_clean['gold_dir'].shift(1)
gold_data_clean.head()

/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a co

,Date,Price_Gold,Day_Gold,Gold_Change%,Gold_Change%_lag,gold_dir,gold_dir_lag
0,2016-09-08,1400.2,Thursday,NaN,NaN,0.0,NaN
1,2016-09-09,1393.1,Friday,-0.005071,NaN,0.0,0.0
2,2016-09-12,1384.4,Monday,-0.006245,-0.005071,0.0,0.0
3,2016-09-13,1380.8,Tuesday,-0.002600,-0.006245,0.0,0.0
4,2016-09-14,1383.3,Wednesday,0.001811,-0.002600,1.0,0.0


In [16]:
gold_data_clean.dtypes

Date                datetime64[ns]
Price_Gold                 float64
Day_Gold                    object
Gold_Change%               float64
Gold_Change%_lag           float64
gold_dir                   float64
gold_dir_lag               float64
dtype: object

In [17]:
snp = pd.read_csv("data/S&P-500.csv")
snp = snp.drop(columns=["Vol.","Change %","Open","High","Low"])
snp['Date']= pd.to_datetime(snp['Date'])
snp_data = snp.dropna()
# snp_data.head()

In [18]:
snp_data = snp_data.rename(columns={'Price': 'Price_Snp'})
snp_data['Date'] = pd.to_datetime(snp_data['Date'])
snp_data['Day_Snp'] = snp_data['Date'].dt.day_name()
snp_data = snp_data.sort_values(by="Date")
snp_data = snp_data.reset_index(drop=True)
snp_data['Price_Snp'] = snp_data.Price_Snp.str.replace(',', '').astype(float)
# snp_data

In [19]:
snp_data_clean = snp_data.loc[((snp_data['Day_Snp'] != "Saturday") & (snp_data['Day_Snp'] != 'Sunday'))]
# snp_data_clean

In [20]:
# snp_data_clean = snp_data_clean.drop(columns=["Day_Snp"])
snp_data_clean["Snp_Change%"] = snp_data_clean["Price_Snp"].pct_change()
snp_data_clean['Snp_Change%_lag'] = snp_data_clean["Snp_Change%"].shift(1)

# Construction direction of S&P movement
snp_data_clean['Snp_dir'] = (snp_data_clean["Snp_Change%"]>0).astype(float)
snp_data_clean['Snp_dir_lag'] = snp_data_clean['Snp_dir'].shift(1)
# snp_data_clean

In [21]:
usbond = pd.read_csv("data/US-10-Year-Bond.csv")
usbond = usbond.drop(columns=["Change %","Open","High","Low"])
usbond['Date']= pd.to_datetime(usbond['Date'])
usbond_data = usbond.dropna()
# usbond_data.head()

In [22]:
usbond_data = usbond_data.rename(columns={'Price': 'Price_USB'})
usbond_data['Date'] = pd.to_datetime(usbond_data['Date'])
usbond_data['Day_USB'] = usbond_data['Date'].dt.day_name()
usbond_data = usbond_data.sort_values(by="Date")
usbond_data = usbond_data.reset_index(drop=True)
# usbond_data

In [23]:
usbond_data_clean = usbond_data.loc[((usbond_data['Day_USB'] != "Saturday") & (usbond_data['Day_USB'] != 'Sunday'))]
# usbond_data_clean

In [24]:
# usbond_data_clean = usbond_data_clean.drop(columns=["Day_USB"])
usbond_data_clean["USB_Change%"] = usbond_data_clean["Price_USB"].pct_change()
usbond_data_clean['USB_Change%_lag'] = usbond_data_clean["USB_Change%"].shift(1)

# Construction direction of US BOND movement
usbond_data_clean['USB_dir'] = (usbond_data_clean["USB_Change%"]>0).astype(float)
usbond_data_clean['USB_dir_lag'] = usbond_data_clean['USB_dir'].shift(1)
# usbond_data_clean

/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a co

In [25]:
usbond_data_clean.dtypes

Date               datetime64[ns]
Price_USB                 float64
Day_USB                    object
USB_Change%               float64
USB_Change%_lag           float64
USB_dir                   float64
USB_dir_lag               float64
dtype: object

In [26]:
bit_1 = btc_data_clean.merge(eth_data_clean, how='left', on='Date')
# bit_1

In [27]:
bit_2 = bit_1.merge(snp_data_clean, how='left', on='Date')
# bit_2

In [28]:
bit_3 = bit_2.merge(usbond_data_clean, how='left', on='Date')
bit_3.dropna().head()

,Date,Close_Btc,Day_Btc,BTC_Change%,BTC_Change%_lag,BTC_dir,BTC_dir_lag,Close_Eth,Day_Eth,ETH_Change%,...,Snp_Change%,Snp_Change%_lag,Snp_dir,Snp_dir_lag,Price_USB,Day_USB,USB_Change%,USB_Change%_lag,USB_dir,USB_dir_lag
2,2016-10-12,636.192017,Wednesday,-0.007612,0.035668,0.0,1.0,11.9335,Wednesday,0.012541,...,0.001147,-0.012447,1.0,0.0,1.771,Wednesday,0.002264,0.001133,1.0,1.0
3,2016-10-13,636.786011,Thursday,0.000934,-0.007612,1.0,0.0,11.9630,Thursday,0.002472,...,-0.003099,0.001147,0.0,1.0,1.746,Thursday,-0.014116,0.002264,0.0,1.0
4,2016-10-14,640.377991,Friday,0.005641,0.000934,1.0,1.0,11.9382,Friday,-0.002073,...,0.000202,-0.003099,1.0,0.0,1.805,Friday,0.033792,-0.014116,1.0,0.0
5,2016-10-17,639.192993,Monday,-0.001850,0.005641,0.0,1.0,12.0050,Monday,0.005595,...,-0.003038,0.000202,0.0,1.0,1.768,Monday,-0.020499,0.033792,0.0,1.0
6,2016-10-18,637.960022,Tuesday,-0.001929,-0.001850,0.0,0.0,12.5885,Tuesday,0.048605,...,0.006160,-0.003038,1.0,0.0,1.740,Tuesday,-0.015837,-0.020499,0.0,0.0


In [29]:
# bit_3.dtypes

In [30]:
gold_1 = gold_data_clean.merge(usbond_data_clean, how='left', on='Date')
# gold_1

In [31]:
gold_2 = gold_1.merge(snp_data_clean, how='left', on='Date')
# gold_2

In [32]:
gold_3 = gold_2.merge(eth_data_clean, how='left', on='Date')
gold_3 = gold_3.dropna()
gold_3 = gold_3.reset_index(drop=True)

gold_3 = gold_3[['Date', 'gold_dir', 'gold_dir_lag', 'Gold_Change%_lag', 'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
gold_3.tail()

,Date,gold_dir,gold_dir_lag,Gold_Change%_lag,USB_Change%_lag,Snp_Change%_lag,ETH_Change%_lag
1245,2021-09-30,1.0,0.0,-0.008403,-0.014230,0.001569,0.016331
1246,2021-10-01,1.0,1.0,0.019792,-0.020997,-0.011910,0.052060
1247,2021-10-04,1.0,1.0,0.000797,-0.018097,0.011494,0.101889
1248,2021-10-05,0.0,1.0,0.005232,0.010922,-0.012988,0.021942
1249,2021-10-06,1.0,0.0,-0.003790,0.031735,0.010524,0.040954


In [33]:
train = gold_3.iloc[:-100, :]
test = gold_3.iloc[-100:, :]
train.tail(), test.head()

x_train = train[['gold_dir_lag', 'Gold_Change%_lag',  'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
y_train = train['gold_dir']
x_test = test[['gold_dir_lag', 'Gold_Change%_lag',  'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
y_test = test['gold_dir']

# x_train, y_train

In [34]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=1000)
classifier

LogisticRegression(max_iter=1000)

In [35]:
classifier = classifier.fit(x_train, y_train)

In [36]:
print(f"Training Data Score: {classifier.score(x_train, y_train)}")
print(f"Testing Data Score: {classifier.score(x_test, y_test)}")

Training Data Score: 0.5539130434782609
Testing Data Score: 0.56


In [37]:
predictions = classifier.predict(x_test)
print(f"First 10 Predictions: {predictions[:10]}")
print(f"First 10 Actual Data: {y_test[:10].tolist()}")

First 10 Predictions: [1. 0. 1. 0. 1. 1. 1. 1. 1. 1.]
First 10 Actual Data: [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0]


In [38]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,1.0,1.0
1,0.0,1.0
2,1.0,1.0
3,0.0,1.0
4,1.0,0.0
...,...,...
95,1.0,1.0
96,1.0,1.0
97,1.0,1.0
98,1.0,0.0


In [39]:
features = x_train.columns.tolist()
selector = RFECV(estimator=classifier, cv=5, step=1)
_ = selector.fit(x_train, y_train)

In [40]:
selected_features = sorted(zip(selector.ranking_, features))
features_rank = pd.DataFrame(selected_features, columns=['Ranking', 'Feature'])
features_rank = features_rank.set_index('Feature')
features_rank

,Ranking
Feature,
ETH_Change%_lag,1
USB_Change%_lag,1
gold_dir_lag,2
Gold_Change%_lag,3
Snp_Change%_lag,4


In [41]:
ranked_features = []
for s in selected_features:
    if s[0] < 3:
        ranked_features.append(s[1])

In [42]:
len(ranked_features)

3

In [43]:
x_train_select = x_train[ranked_features]
x_test_select = x_test[ranked_features]

In [44]:
classifier_1 = LogisticRegression(solver='newton-cg', multi_class='auto')
classifier_1.fit(x_train_select, y_train)

print(f"Training Data Score: {classifier_1.score(x_train_select, y_train)}")
print(f"Testing Data Score: {classifier_1.score(x_test_select, y_test)}")

Training Data Score: 0.5547826086956522
Testing Data Score: 0.56


# Hyperparameter Tuning

In [45]:
classifier_2 = LogisticRegression(multi_class='auto')

param_grid = {'penalty': ["l1", "l2", 'elasticnet'],
              'solver': ['newton-cg','lbfgs','saga'],
             'C': [1,10,100]}
grid = GridSearchCV(classifier_2, param_grid, n_jobs=-2)

In [46]:
_ = grid.fit(x_train_select, y_train)

/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.54782609 0.54608696 0.54608696 0.54608696
        nan        nan        nan        nan        nan 0.54956522
 0.55217391 0.55217391 0.55217391        nan        nan        nan
        nan        nan 0.55043478 0.55043478 0.55043478 0.55043478
        nan        nan        nan]
  category=UserWarning


In [47]:
penalty = grid.best_params_['penalty']
solver = grid.best_params_['solver']
C = grid.best_params_['C']

#tuned Model

model_tuned = LogisticRegression(multi_class='auto', penalty=penalty, solver=solver, C=C)
model_tuned.fit(x_train_select, y_train)

classifier_2_training_score = round(model_tuned.score(x_train_select, y_train)*100,3)
classifier_2_test_score = round(model_tuned.score(x_test_select, y_test)*100,3)

print(f"Training Data Score: {classifier_2_training_score} %")
print(f"Testting Data Score: {classifier_2_test_score} %")

Training Data Score: 55.478 %
Testting Data Score: 54.0 %


# Randon Forest

In [48]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
param_grid = {'max_features':['auto', 'sqrt', 'log2'],
           'class_weight':['balanced', 'balanced_subsample'],
             'criterion':['gini', 'entropy']}

grid = GridSearchCV(rf, param_grid, n_jobs=-1)

In [49]:
_ = grid.fit(x_train_select, y_train)

In [50]:
max_features = grid.best_params_['max_features']
class_weight = grid.best_params_['class_weight']
criterion = grid.best_params_['criterion']

In [51]:
tuned_model = RandomForestClassifier(n_estimators=1000, max_features=max_features, class_weight=class_weight, criterion=criterion, random_state=42)
tuned_model.fit(x_train_select, y_train)
rf_training_score = round(tuned_model.score(x_train_select, y_train)*100,3)
rf_test_score = round(tuned_model.score(x_test_select, y_test)*100,3)

print(f"Training Data Score: {rf_test_score} %")
print(f"Testting Data Score: {rf_test_score} %")

Training Data Score: 59.0 %
Testting Data Score: 59.0 %


In [52]:
# sorted(zip(grid.feature_importances_, ranked_features), reverse=True)

In [53]:
print("n\tscore")
print("-"*20)
for n in [5,10,50,100,200,400,500,700,1000]:
    rf = RandomForestClassifier(n_estimators=n, random_state=42)
    rf = rf.fit(x_train_select, y_train)
    print(n,"\t",rf.score(x_test_select, y_test))

n	score
--------------------
5 	 0.49
10 	 0.51
50 	 0.53
100 	 0.56
200 	 0.55
400 	 0.57
500 	 0.57
700 	 0.57
1000 	 0.58


In [54]:
from sklearn.ensemble import RandomForestClassifier
rfr = RandomForestClassifier(n_estimators=1000, random_state=22)
rfr = rfr.fit(x_train_select, y_train)
rfr.score(x_test_select, y_test)

0.59

In [55]:
from sklearn.linear_model import LinearRegression
model_l = LinearRegression()

In [56]:
model_l.fit(x_train, y_train)

training_score = model_l.score(x_train, y_train)
testing_score = model_l.score(x_test, y_test)
print(f"R2 Score: {training_score}")
print(f"Testing Score: {testing_score}")

R2 Score: 0.011510179142137367
Testing Score: 0.013710935391888834


In [57]:
from sklearn.svm import SVC
model_svc = SVC(kernel='linear',)
model_svc = model_svc.fit(x_train, y_train)

In [58]:
print(f"Training Score: {model_svc.score(x_train, y_train)}")
print(f"Testing Score: {model_svc.score(x_test,y_test)}")

Training Score: 0.5495652173913044
Testing Score: 0.57


In [59]:
predictions = model_svc.predict(x_test)
print(f"First 10 Predictions: {predictions[:-10]}")
print(f"First 10 Actual Data: {y_test[:10].tolist()}")

First 10 Predictions: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
First 10 Actual Data: [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0]


In [60]:
from numpy import loadtxt
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score

In [61]:
# xg_model = XGBClassifier()
xg_model = XGBRegressor()
xg_model.fit(x_train, y_train, eval_metric="logloss", verbose=True)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=1,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [62]:
xg_predict = xg_model.predict(x_test)
# xg_model.score(x_test, y_test)
# xg_predit = xg_predict.tolist()
xg_predict = pd.Series(xg_predict)
xg_predict = (xg_predict>0.5).astype(float)
xg_predict

0     0.0
1     1.0
2     0.0
3     0.0
4     0.0
     ... 
95    1.0
96    1.0
97    0.0
98    0.0
99    1.0
Length: 100, dtype: float64

In [63]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, xg_predict)

array([[18, 25],
       [25, 32]])

In [64]:
model_svc.predict(x_test)

confusion_matrix(y_test, model_svc.predict(x_test))

array([[ 0, 43],
       [ 0, 57]])

In [65]:

confusion_matrix(y_test, rfr.predict(x_test_select))

array([[20, 23],
       [18, 39]])

In [66]:
confusion_matrix(y_test, model_tuned.predict(x_test_select))

array([[ 7, 36],
       [10, 47]])

In [67]:
type(rfr.predict(x_test_select))
y_test2 = y_test.reset_index(drop=True)
data = {'True_Value': y_test2.tolist(), 'XG_pred': xg_predict.tolist(), 'RF_pred': rfr.predict(x_test_select).tolist() }
pred_df = pd.DataFrame(data=data)
pred_df.head()   

pred_df['trade'] = np.where((pred_df['XG_pred']==0) & (pred_df['RF_pred']==0), -1, 
                            np.where((pred_df['XG_pred']==1) & (pred_df['RF_pred']==1), 1, 0))
pred_df

pred_df[(pred_df['True_Value']==1) & (pred_df['trade']==1)].count()

True_Value    26
XG_pred       26
RF_pred       26
trade         26
dtype: int64

In [68]:
pred_df[(pred_df['True_Value']==0) & (pred_df['trade']==0)].count()

True_Value    18
XG_pred       18
RF_pred       18
trade         18
dtype: int64

In [69]:
pred_df[(pred_df['True_Value']==1) & (pred_df['trade']==-1)].count()

True_Value    12
XG_pred       12
RF_pred       12
trade         12
dtype: int64

In [70]:
pred_df[(pred_df['True_Value']==0) & (pred_df['trade']==1)].count()

True_Value    15
XG_pred       15
RF_pred       15
trade         15
dtype: int64

In [71]:
(26+18)/(26+18+12+15)

0.6197183098591549

In [72]:
# pred_xg = pd.DataFrame(columns=['Date', 'gold_dir', 'XG'])

# days_xg = 150
prediction_window = 130
for days_xg in range(10, 1000,10):
    pred_xg = pd.DataFrame(columns=['Date', 'gold_dir', 'XG'])


    data_xg = gold_3.iloc[-days_xg-prediction_window:, :]

    for i in range(0, prediction_window):
        train = data_xg.iloc[i:days_xg+i, :] # Training model
        test = data_xg.iloc[days_xg+i : days_xg+i+1, :]
    #     print(train.tail(2), test)

        x_train = train.drop(columns=['gold_dir', 'Date'])
        y_train = train['gold_dir']

        x_test = test.drop(columns=['gold_dir', 'Date'])
        y_test = test['gold_dir']

        xg_m = XGBRegressor()
        xg_m2 = xg_m.fit(x_train, y_train)
        y_pred_xg = xg_m2.predict(x_test)

        df_reg = pd.DataFrame({'XG': y_pred_xg.tolist()})
        true_data = test[['Date', 'gold_dir']]
        true_data = true_data.reset_index(drop=True)

        df = pd.concat([true_data, df_reg], axis=1)
        pred_xg = pred_xg.append(df)    

    #     break

    pred_xg['xg_pred'] = (pred_xg['XG']>0.5).astype(float)
    a_xg = confusion_matrix(pred_xg['gold_dir'], pred_xg['xg_pred'])
    print(days_xg, "\n", a_xg)
    print("accuracy")
    print(round ((a_xg[0][0] + a_xg[1][1]) / (a_xg[0][0]+ a_xg[1][1] + a_xg[0][1] + a_xg[1][0]), 4))

10 
 [[21 36]
 [41 32]]
accuracy
0.4077
20 
 [[21 36]
 [35 38]]
accuracy
0.4538
30 
 [[21 36]
 [34 39]]
accuracy
0.4615
40 
 [[25 32]
 [33 40]]
accuracy
0.5
50 
 [[20 37]
 [34 39]]
accuracy
0.4538
60 
 [[28 29]
 [40 33]]
accuracy
0.4692
70 
 [[24 33]
 [36 37]]
accuracy
0.4692
80 
 [[25 32]
 [38 35]]
accuracy
0.4615
90 
 [[28 29]
 [38 35]]
accuracy
0.4846
100 
 [[25 32]
 [40 33]]
accuracy
0.4462
110 
 [[27 30]
 [33 40]]
accuracy
0.5154
120 
 [[27 30]
 [33 40]]
accuracy
0.5154
130 
 [[23 34]
 [38 35]]
accuracy
0.4462
140 
 [[29 28]
 [34 39]]
accuracy
0.5231
150 
 [[24 33]
 [35 38]]
accuracy
0.4769
160 
 [[25 32]
 [32 41]]
accuracy
0.5077
170 
 [[26 31]
 [30 43]]
accuracy
0.5308
180 
 [[22 35]
 [32 41]]
accuracy
0.4846
190 
 [[22 35]
 [33 40]]
accuracy
0.4769
200 
 [[30 27]
 [38 35]]
accuracy
0.5
210 
 [[26 31]
 [37 36]]
accuracy
0.4769
220 
 [[27 30]
 [33 40]]
accuracy
0.5154
230 
 [[32 25]
 [33 40]]
accuracy
0.5538
240 
 [[30 27]
 [30 43]]
accuracy
0.5615
250 
 [[26 31]
 [31 42]]
accura

In [73]:
xgc_model = XGBClassifier()
xgc_model.fit(x_train, y_train)

/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=1,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [74]:
xgc_model.score(x_test, y_test)

0.0

In [76]:
!pip install statsmodels

  Using cached statsmodels-0.12.2-cp36-cp36m-macosx_10_15_x86_64.whl (9.5 MB)
     |████████████████████████████████| 233 kB 5.7 MB/s eta 0:00:01


In [78]:
#install these packages
from sklearn.decomposition import PCA
import statsmodels.tools.tools as sm
from sklearn.metrics import confusion_matrix

In [81]:
# code
buffer=132
for days_rf in range(10, 900,10):
    pred_rf = pd.DataFrame(columns=['Date', 'gold_dir', 'rs', 'rf'])
    data_rf = gold_3.iloc[-days_rf-buffer:, :]
    
    for rs in range(1, 31):   # Various random states
        for i in range(0, buffer): # Rolling window prediction
            train = data_rf.iloc[i:days_rf+i, :] # Training model
            test = data_rf.iloc[days_rf+i : days_rf+i+1, :]
#             print(train.tail(2), test)

            # Separating X and Y variables for easy computation
            X_train = train.drop(columns=['Date', 'gold_dir'])
            X_test = test.drop(columns=['Date', 'gold_dir'])

            y_train = train['gold_dir']
            y_test = test['gold_dir']

            pca = PCA(n_components = 0.95).fit(X_train) # Principal Component Analysis

            train_pca = pca.transform(X_train)
            test_pca = pca.transform(X_test)

            # Random Forest Model
            random_forest = RandomForestClassifier(oob_score= False, n_estimators= 50, min_samples_split= 2, max_features= 'log2',
                                    criterion= 'gini', bootstrap= True, random_state=rs)
#             random_forest = RandomForestClassifier()
            rf_model = random_forest.fit(train_pca, y_train) 
            y_pred_rf = rf_model.predict(test_pca)

            ls_rs = [rs]*len(y_test)

            df_reg = pd.DataFrame({'rs': ls_rs, 'rf': y_pred_rf.tolist()})
            true_data = test[['Date', 'gold_dir']]
            true_data = true_data.reset_index().drop(columns=['index'])
            df = pd.concat([true_data, df_reg], axis=1)
            pred_rf = pred_rf.append(df)
#             print(pred_rf.tail(2))

        # Retaining only that random state with highest accuracy      
        for i in range(1,pred_rf.shape[1]): # The outputs may come in str type  
            pred_rf.iloc[:,i] = pred_rf.iloc[:,i].astype(float)

        cm = pd.DataFrame(columns=['RF CM', 'RF Accuracy'])

        # Estimating Confusion matrix for each model and random state
    for rs in range(1, 31):
        df = pred_rf[pred_rf['rs']==rs]
#             prin
        a_rf = (confusion_matrix(df['gold_dir'], df['rf']))
        b_rf = round((a_rf[0][0] + a_rf[1][1])/(a_rf[0][0]+ a_rf[1][1] + a_rf[0][1] + a_rf[1][0]),4) 
        conf_mat = pd.DataFrame({'RF CM': [a_rf], 'RF Accuracy': [b_rf]}) # Appending to dataframes element-wise
        cm = cm.append(conf_mat)

    # Constructing index such that it refers to random state, which is later named as 'rs'   
    cm = cm.reset_index().drop(columns='index')
    cm.index += 1
    cm = cm.reset_index().rename(columns={'index':'rs'})
    max_acc_rs = cm['rs'][cm['RF Accuracy']==cm['RF Accuracy'].max()] # Retaining obs with highest accuracy

    pred_rf_max_acc = pred_rf[pred_rf['rs']==max_acc_rs.iloc[0]]
    print('days_rf:', days_rf, cm[cm['rs']==max_acc_rs.iloc[0]])

days_rf: 10    rs                 RF CM  RF Accuracy
5   6  [[20, 37], [28, 47]]       0.5076
days_rf: 20    rs                 RF CM  RF Accuracy
5   6  [[26, 31], [32, 43]]       0.5227
days_rf: 30     rs                 RF CM  RF Accuracy
17  18  [[23, 34], [27, 48]]       0.5379
days_rf: 40    rs                 RF CM  RF Accuracy
7   8  [[21, 36], [34, 41]]       0.4697
days_rf: 50    rs                 RF CM  RF Accuracy
4   5  [[19, 38], [39, 36]]       0.4167
days_rf: 60    rs                 RF CM  RF Accuracy
5   6  [[23, 34], [43, 32]]       0.4167
days_rf: 70    rs                 RF CM  RF Accuracy
2   3  [[22, 35], [41, 34]]       0.4242
days_rf: 80     rs                 RF CM  RF Accuracy
10  11  [[30, 27], [39, 36]]          0.5
days_rf: 90    rs                 RF CM  RF Accuracy
6   7  [[15, 42], [39, 36]]       0.3864
days_rf: 100    rs                 RF CM  RF Accuracy
1   2  [[22, 35], [35, 40]]       0.4697
days_rf: 110    rs                 RF CM  RF Accuracy
8

days_rf: 870     rs                 RF CM  RF Accuracy
27  28  [[22, 35], [41, 34]]       0.4242
days_rf: 880     rs                 RF CM  RF Accuracy
19  20  [[18, 39], [35, 40]]       0.4394
days_rf: 890     rs                 RF CM  RF Accuracy
22  23  [[23, 34], [31, 44]]       0.5076
